In [18]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

img_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])
train_data = datasets.ImageFolder(root = "train_images", transform = img_transform) 
#TODO Make this an actual dataset
test_data = datasets.ImageFolder(root = "train_images", transform = img_transform) 

loaders = {
    "train" : DataLoader(train_data,
                        batch_size = 100,
                        shuffle = True,
                        num_workers = 1),
    "test"  : DataLoader(test_data,
                        batch_size = 100,
                        shuffle = True,
                        num_workers = 1)
}

In [19]:
train_data

Dataset ImageFolder
    Number of datapoints: 1155
    Root location: train_images
    StandardTransform
Transform: Compose(
               Resize(size=(128, 128), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )

In [12]:
from PIL import Image
img = Image.open("train_images\\five\\five-frame0.jpg")
print(img.size)

(640, 480)


In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class HandRecognizer(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(18000, 6000)
        self.fc2 = nn.Linear(6000, 1000)
        self.fc3 = nn.Linear(1000, 100)
        self.fc4 = nn.Linear(100, 3)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x), 2)))
        x = x.view(-1, 18000)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training = self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training = self.training)
        x = F.relu(self.fc3(x))
        x = F.dropout(x, training = self.training)
        x = self.fc4(x)

        return F.softmax(x, dim = 1)



In [21]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HandRecognizer().to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.001)
loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for num_batch, (data, label) in enumerate(loaders["train"]):
        data, label = data.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, label)
        loss.backward()
        optimizer.step()
        if num_batch % 20 == 0:
            print(f"Train epoch: {epoch} [{num_batch * len(data)}/{len(loaders['train'].dataset)} ({100. * num_batch / len(loaders['train']):.0f}%)]\t{loss.item():.6f}")





In [ ]:
for epoch in range(1, 11):
    model.train()